In [ ]:
### Version 1 - SIMPLE AS OF FEB, 24TH 2026###
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


# URL
url = "https://immovlan.be/en"

driver = webdriver.Safari()
wait = WebDriverWait(driver, 5)

driver.get(url)
time.sleep(3)

# COOKIES
try:
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    cookie_button.click()
except:
    pass

# ZIP
location_input = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//input[@placeholder='Where? City, ZipCode, Province or Region']")
    )
)
zip_code = 1030
location_input.send_keys(zip_code)

# ZIP presented
first_item = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="autocomplete-chip-item-0-0"]'))
)
first_item.click()

# Pressing the SEARCH THE LIST button
search_button = wait.until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.search-list-button"))
)
search_button.click()

# LOADING THE FIRST PAGE WITH RESULTS
wait.until(
    EC.presence_of_element_located((By.XPATH, "//article[@itemscope and @itemtype]"))
)

# GETTING THE URL OF THE CURRENT PAGE
base_url = driver.current_url

# CHECKING IF THERE IS A PARAMETER page=
if "page=" not in base_url:
    separator = "&" if "?" in base_url else "?"
    base_url = f"{base_url}{separator}page="

print("Base URL:", base_url)

# COLLECTING LINKS OF LOTS
lots_urls = []

for page in range(1, 51):
    page_url = base_url + str(page)
    try:
        # GOING TO THE PAGE
        driver.get(page_url)
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@itemscope and @itemtype]")))

        print(f"\nScraping page {page}")

        lots = driver.find_elements(By.XPATH, "//article[@itemscope and @itemtype]")

        print(f"Lots found on page {page}: {len(lots)}")
        #COLLECTING LINKS
        for lot in lots:
            url = lot.get_attribute("data-url")
            if url and url not in lots_urls:
                lots_urls.append(url)
                
        print(f"Collected so far: {len(lots_urls)}")
    except: # IF NO MORE page_url = base_url + str(page)
        print(f"No more pages with data")
        break
    
print("\nTotal lots collected:", len(lots_urls))

# PRINTING IN TERMINAL


for url in lots_urls:
    print(url)

driver.quit()

# SAVING RESULTS IN CSV
with open("../data/lots_urls.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for url in lots_urls:
        writer.writerow([url])

print(f"Total lots collected: {len(lots_urls)}")
print(f"n\Data saved in 'lots_urls.csv")





<>:107: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
<>:107: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
/var/folders/6j/wxn98q7n083dpm833pdmj1v80000gn/T/ipykernel_95620/4186145357.py:107: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
  print(f"n\Data saved in 'lots_urls.csv")


Base URL: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&towns=1030-schaarbeek&noindex=1&page=

Scraping page 1
Lots found on page 1: 20
Collected so far: 20

Scraping page 2
Lots found on page 2: 20
Collected so far: 40

Scraping page 3
Lots found on page 3: 20
Collected so far: 60

Scraping page 4
Lots found on page 4: 20
Collected so far: 80

Scraping page 5
Lots found on page 5: 20
Collected so far: 100

Scraping page 6
Lots found on page 6: 20
Collected so far: 120

Scraping page 7
Lots found on page 7: 20
Collected so far: 140

Scraping page 8
Lots found on page 8: 20
Collected so far: 160

Scraping page 9
Lots found on page 9: 20
Collected so far: 180

Scraping page 10
Lots found on page 10: 20
Collected so far: 200

Scraping page 11
Lots found on page 11: 20
Collected so far: 220

Scraping page 12
Lots found on page 12: 20
Collected so far: 240

Scraping page 13
Lots found on page 13: 20
Collected so far: 260

Scraping page 14
Lots found on page 14:

In [ ]:
### Version 2 - SIMPLIFIED ###
# USING SELENIUM TO GO THRU THE COOKIES, SELECT THE PROPERTY LOT TYPE (House, Apartment) AND FIRST PROVINCE TO MAKE A PATTERN
# USE REGULAR MANIPULATION TO WORK WITH URL LINK AND GET THE DATA FOR ALL PROVINCES 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


# URL
url = "https://immovlan.be/en"


# IMPORTANT: You need to choose your correct browser to run.  I did it with Safari browser
driver = webdriver.Safari()
wait = WebDriverWait(driver, 5)

driver.get(url)
time.sleep(3)

# COOKIES
try:
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    cookie_button.click()
except:
    pass

# Choosing 'House' and 'Apartment' to narrow the search
driver.find_element(By.XPATH, "//input[@placeholder='What?']").click()
time.sleep(3)
driver.find_element(By.XPATH, "//span[contains(text(), 'House')]").click()
driver.find_element(By.XPATH, "//span[contains(text(), 'Apartment')]").click()
time.sleep(1)

# Creating the list of provinces of Belgium from the site
ul = driver.find_element(By.ID, "button-group-spotlight-province")
links = ul.find_elements(By.TAG_NAME, "a")
provinces_list = [link.text.strip() for link in links]

print("Provinces list:", provinces_list)

province_to_select = provinces_list[0]  # Select the first province aka Brussels for testing

# Applying the search pattern for the province selection
# Finding the input field for the location
location_input = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//input[@placeholder='Where? City, ZipCode, Province or Region']")
    )
)
# Slowly typing the province name to trigger the dropdown suggestions    
for ch in province_to_select: 
    location_input.send_keys(ch)
    time.sleep(0.15)
# Waiting for the dropdown suggestions to appear and selecting the correct province
    items = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li[role='option']"))
    )
    for item in items:
        value_type = item.get_attribute("data-value-type")
        text = item.text.strip()
        if value_type == "province" and province_to_select.lower() in text.lower():
            item.click()
            break
    

# Pressing the 'SEARCH THE LIST' button
search_button = wait.until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.search-list-button"))
)
search_button.click()

# LOADING THE FIRST PAGE WITH RESULTS
wait.until(
    EC.presence_of_element_located((By.XPATH, "//article[@itemscope and @itemtype]"))
)

# GETTING THE URL OF THE CURRENT PAGE
base_province_url = driver.current_url
print(f"Base url before changing pages and provinces: {base_province_url}")

# INITIALIZING THE LIST OF LOTS URLS
lots_urls = []

# THE LOOP TO SCRAPE LOTS LINKS FOR EACH PROVINCE
# We will replace the province parameter in the URL to navigate through different provinces without having to interact with the page elements again
for province in provinces_list:
    province_url = base_province_url.replace("provinces=brussels", f"provinces={province.lower()}")
    print(f"New URL for {province}: {province_url}")
    

    # CHECKING IF THERE IS A PARAMETER page=
    if "page=" not in province_url:
        separator = "&" if "?" in province_url else "?"
        base_url = f"{province_url}{separator}page="

    print("Base URL:", base_url)

    # COLLECTING LINKS OF LOTS FOR THE CURRENT PROVINCE, MAX PAGE = 50 - NO MORE ALLOWED BY THE SITE
    for page in range(1, 51):
        page_url = base_url + str(page)
        try:
            # GOING TO THE PAGE
            driver.get(page_url)
            wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@itemscope and @itemtype]")))

            print(f"\nScraping page {page}")

            # A LIST OF ALL LOTS ON THE PAGE
            lots = driver.find_elements(By.XPATH, "//article[@itemscope and @itemtype]")

            print(f"Lots found on page {page}: {len(lots)}")

            #COLLECTING LINKS
            for lot in lots:
                url = lot.get_attribute("data-url")
                if url and url not in lots_urls:
                    lots_urls.append(url)
                    
            print(f"Collected so far: {len(lots_urls)}")
        except: # IF NO MORE page_url = base_url + str(page)
            print(f"No more pages with data")
            break
    
print("\nTotal lots collected:", len(lots_urls))

# PRINTING IN TERMINAL TO CHECK
for url in lots_urls:
    print(url)

driver.quit()

# SAVING RESULTS IN CSV
with open("../data/lots_urls.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for url in lots_urls:
        writer.writerow([url])

print(f"Total lots collected: {len(lots_urls)}")
print(f"Data saved in 'lots_urls.csv")



<>:140: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
<>:140: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
/var/folders/6j/wxn98q7n083dpm833pdmj1v80000gn/T/ipykernel_95620/4037394602.py:140: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
  print(f"n\Data saved in 'lots_urls.csv")


Provinces list: ['Brussels', 'Hainaut', 'Liège', 'East Flanders', 'West Flanders', 'Antwerp', 'Vlaams-Brabant', 'Brabant Wallon', 'Limburg', 'Luxembourg', 'Namur']
Base url before changing pages and provinces: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces=brussels&noindex=1
New URL for Brussels: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces=brussels&noindex=1
Base URL: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces=brussels&noindex=1&page=

Scraping page 1
Lots found on page 1: 20
Collected so far: 20

Scraping page 2
Lots found on page 2: 20
Collected so far: 40

Scraping page 3
Lots found on page 3: 20
Collected so far: 60

Scraping page 4
Lots found on page 4: 20
Collected so far: 80

Scraping page 5
Lots found on page 5: 20
Collected so far: 100

Scraping page 6
Lots found on 

In [38]:
checking_nonduplicates = {l for l in lots_urls}
print(f"Total nonduplicates found: {len(checking_nonduplicates)}")
new_set = set(lots_urls)
print(f"Total unique lots collected: {len(new_set)}")

Total nonduplicates found: 10979
Total unique lots collected: 10979


In [2]:
import csv
with open("../data/lots_urls.csv", "r", newline="") as f:
    reader = csv.reader(f)
    csv_lots_urls = [row[0] for row in reader]
    my_set = set(csv_lots_urls)

with open("../data/all_properties_urls_without_projects.csv", "r", newline="") as f:
    reader = csv.reader(f)
    csv_lots_urls = [row[0] for row in reader]
    Simon_set = set(csv_lots_urls)

only_in_mine = my_set - Simon_set
only_in_Simon = Simon_set - my_set
print(f"Total lots only in my set: {len(only_in_mine)}")
print(f"Total lots only in Simon's set: {len(only_in_Simon)}")



Total lots only in my set: 925
Total lots only in Simon's set: 3186


In [3]:
for url in only_in_mine:
    print(url)

https://immovlan.be/en/projectdetail/25733-4102767
https://immovlan.be/en/projectdetail/21575-5381227
https://immovlan.be/en/projectdetail/2278297-rmg01398585_om_5809
https://immovlan.be/en/projectdetail/25923-7323880
https://immovlan.be/en/projectdetail/23984-3708797
https://immovlan.be/en/projectdetail/1485772-01267922_om_293966
https://immovlan.be/en/projectdetail/1865181-5277835
https://immovlan.be/en/detail/apartment/for-sale/1081/koekelberg/vbd86143
https://immovlan.be/en/projectdetail/23976-639539d8-5792-4e50-8aa8-433e606ac8b0
https://immovlan.be/en/detail/apartment/for-sale/9160/lokeren/rbv35047
https://immovlan.be/en/projectdetail/1290232-3701078
https://immovlan.be/en/projectdetail/21830-6209563
https://immovlan.be/en/projectdetail/vi0002162
https://immovlan.be/en/detail/apartment/for-sale/1083/ganshoren/vbd89461
https://immovlan.be/en/projectdetail/1901438-6060186
https://immovlan.be/en/detail/residence/for-sale/6042/lodelinsart/vbd90893
https://immovlan.be/en/projectdetail/

In [4]:
for url in only_in_Simon:
    print(url)
    

https://immovlan.be/en/detail/apartment/for-sale/1200/sint-lambrechts-woluwe/vbd63613
https://immovlan.be/en/detail/apartment/for-sale/1050/elsene/vwd15795
https://immovlan.be/en/detail/penthouse/for-sale/1160/oudergem/vbc64806
https://immovlan.be/en/detail/apartment/for-sale/1070/anderlecht/vbd26569
https://immovlan.be/en/detail/mixed-building/for-sale/1200/sint-lambrechts-woluwe/vbd64794
https://immovlan.be/en/detail/apartment/for-sale/1200/sint-lambrechts-woluwe/vba02913
https://immovlan.be/en/detail/apartment/for-sale/1140/evere/vba21335
https://immovlan.be/en/detail/apartment/for-sale/1070/anderlecht/vbd35133
https://immovlan.be/en/detail/duplex/for-sale/1210/sint-joost-ten-node/vbd68932
https://immovlan.be/en/detail/duplex/for-sale/1020/laken/vbd75878
https://immovlan.be/en/detail/apartment/for-sale/1000/brussels/vbd44719
https://immovlan.be/en/detail/apartment/for-sale/1070/anderlecht/vbb88794
https://immovlan.be/en/detail/studio/for-sale/1040/etterbeek/vbd79485
https://immovlan

In [2]:
### TESTING SELENIUM @ FEB 26TH, 2026 ###

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import unicodedata

# URL
url = "https://immovlan.be/en"

driver = webdriver.Safari()
wait = WebDriverWait(driver, 10)

# RUNNING DRIVER
driver.get(url)
time.sleep(3)

# COOKIES
try:
    cookie_button = wait.until(
        EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
    )
    cookie_button.click()
except:
    pass
time.sleep(3)

# # MAXIMALIZING THE WINDOW TO AVOID ISSUES WITH ELEMENTS NOT BEING CLICKABLE
# driver.fullscreen_window()
# time.sleep(1)
# driver.maximize_window()
# time.sleep(5)

# FUNCTION TO SELECT THE PROPERTY TYPE (House, Apartment) TO NARROW THE SEARCH
def select_property_type(driver):
    driver.find_element(By.XPATH, "//input[@placeholder='What?']").click()
    time.sleep(3)
    driver.find_element(By.XPATH, "//span[contains(text(), 'House')]").click()
    driver.find_element(By.XPATH, "//span[contains(text(), 'Apartment')]").click()
    time.sleep(1)

# RUNNING IT TO GET THE ERASE OPTIONS BUTTON TO APPEAR AND AVOID ISSUES WITH THE FILTERS
select_property_type(driver)

# CREATING LIST OF AREAS TO SEARCH = PROVINCES + MUNICIPALITIES OF BRUSSELS
ul = driver.find_element(By.ID, "button-group-spotlight-province")
links = ul.find_elements(By.TAG_NAME, "a")
provinces_list = [link.text.strip() for link in links]

# chosen = ['Brussels']

bx_municipalities=["1000-brussels","1020-laken","1030-schaarbeek","1040-etterbeek","1050-elsene","1060-sint-gillis","1070-anderlecht","1080-sint-jans-molenbeek","1090-jette","1120-neder-over-heembeek","1130-haren","1140-evere","1150-sint-pieters-woluwe","1160-oudergem","1170-watermaal-bosvoorde","1180-ukkel","1190-vorst","1200-sint-lambrechts-woluwe","1210-sint-joost-ten-node"]

area_list = provinces_list[1:] + bx_municipalities
print(f"Area list to search: {area_list}")


# FUNCTION TO NORMALIZE THE PROVINCE NAMES TO MATCH THE SLUGS IN THE HTML ATTRIBUTES
def normalize_slug(text):
    # "Liège" → "Liege"
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return text.lower().replace(" ", "-")


# FUNCTION TO REMOVE THE APPLIED FILTER
def remove_selected_filter(driver): 
    driver.find_element(By.XPATH, "//a[contains(text(), 'Erase options')]").click()
time.sleep(1)

lots_urls = []

# FUNCTION TO GET LOTS LINKS FOR THE PAGE
def get_lots_links_for_page(page_url, lots_urls):
    try:
        # GOING TO THE PAGE
        driver.get(page_url)
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@itemscope and @itemtype]")))

        # A LIST OF ALL LOTS ON THE PAGE
        lots = driver.find_elements(By.XPATH, "//article[@itemscope and @itemtype]")

        #COLLECTING LINKS
        for lot in lots:
            url = lot.get_attribute("data-url")
            if url and url not in lots_urls:
                lots_urls.append(url)
                
        print(f"Collected so far: {len(lots_urls)}")
    except:
        print(f"No more pages with data")
    
    return lots_urls

# FUNCTION TO SELECT THE AREA (PROVINCE OR MUNICIPALITY) TO NARROW THE SEARCH
def select_area(driver, area):
    if area[:4].isdigit():
        mode = 'city'
        area_fixed = area[:4]  # For Brussels municipalities, we take only the first 4 characters to match the city code
    else:
        mode = 'province'
        area_fixed = area

    # CLEARING THE INPUT FIELD
    input_box = wait.until(
        EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Where? City, ZipCode, Province or Region']"))
    )
    input_box.clear()
    time.sleep(0.3)

    # INPUTTING THE AREA NAME
    input_box.send_keys(area_fixed)
    time.sleep(0.5)

    # WAITING FOR THE DROPDOWN SUGGESTIONS TO APPEAR
    wait.until(EC.presence_of_element_located((By.XPATH, "//li[@role='option']")))

    # CREATING A SLUG TO FIND THE CORRECT ELEMENT IN THE DROPDOWN
    slug = normalize_slug(area)

    xpath = f"//li[@role='option' and @data-value-type='{mode}' and @data-value-slug='{slug}']"

    # WAITING FOR THE CORRECT OPTION TO BE PRESENT IN THE DROPDOWN
    option = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    # CLICKING WITH JS (Safari fix)
    driver.execute_script("arguments[0].click();", option)

# FINALLY...
for area in area_list:

    # REMOVING THE PREVIOUSLY APPLIED FILTER
    try:
        remove_selected_filter(driver)
    except:
        pass

    # SELECTING THE PROPERTY TYPE AGAIN TO AVOID ISSUES WITH THE FILTERS
    select_property_type(driver)

    # SETTING THE AREA
    select_area(driver, area)
    
    time.sleep(2)

    # PRESSING THE 'SEARCH THE LIST' BUTTON
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.search-list-button"))).click()

    # LOADING THE FIRST PAGE WITH RESULTS
    wait.until(EC.presence_of_element_located((By.XPATH, "//article[@itemscope and @itemtype]")))

    # GETTING THE URL OF THE CURRENT PAGE
    base_area_url = driver.current_url
    print(f"Base url for the {area}: {base_area_url}")

    # LOOPING THRU THE PAGES FOR THE CURRENT AREA
    for page in range(1, 3):
        page_url = base_area_url.replace("noindex=1", f"page={page}&noindex=1")
        
        print(f"Page with lots: {page_url}")
        
        lots_urls = get_lots_links_for_page(page_url, lots_urls)


    # CLICKING ON THE LOGO TO GO BACK TO THE HOMEPAGE AND START A NEW SEARCH
    driver.find_element(By.XPATH, "//a[@href='/en']/img[@id='logo']").click()
    time.sleep(5)

    # CONTINUE MY PREVIOUS SEARCH BUTTON CLOSE - CRAZY POP-UP THAT APPEARS RANDOMLY AND BLOCKS THE INTERACTION WITH THE PAGE
    try:
        close_btn = driver.find_element(By.XPATH, "//div[@id='welcome-back-modal']//div[@class='modal-header']//button[contains(@class,'btn-close')]")
        driver.execute_script("arguments[0].click();", close_btn)
        time.sleep(5)
    except:
        pass

driver.quit()

# SAVING RESULTS IN CSV
with open("../data/lots_urls.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for url in lots_urls:
        writer.writerow([url])

print(f"Total lots collected: {len(lots_urls)}")
print(f"Data saved in 'data/lots_urls.csv")

print(lots_urls)

Area list to search: ['Hainaut', 'Liège', 'East Flanders', 'West Flanders', 'Antwerp', 'Brabant Wallon', 'Vlaams-Brabant', 'Namur', 'Limburg', 'Luxembourg', '1000-brussels', '1020-laken', '1030-schaarbeek', '1040-etterbeek', '1050-elsene', '1060-sint-gillis', '1070-anderlecht', '1080-sint-jans-molenbeek', '1090-jette', '1120-neder-over-heembeek', '1130-haren', '1140-evere', '1150-sint-pieters-woluwe', '1160-oudergem', '1170-watermaal-bosvoorde', '1180-ukkel', '1190-vorst', '1200-sint-lambrechts-woluwe', '1210-sint-joost-ten-node']
Base url for the Hainaut: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces=hainaut&noindex=1
Page with lots: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces=hainaut&page=1&noindex=1
Collected so far: 20
Page with lots: https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&pro